In [2]:
import os
import ee
import geemap
from ipyleaflet import Map, basemaps, WidgetControl, Marker, projections
import json
import geopandas as gpd
import ee
import folium
import numpy as np
ee.Initialize()

In [3]:
area = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              21.679458618164062,
              40.41375745020255
            ],
            [
              21.83704376220703,
              40.41375745020255
            ],
            [
              21.83704376220703,
              40.52032411158934
            ],
            [
              21.679458618164062,
              40.52032411158934
            ],
            [
              21.679458618164062,
              40.41375745020255
            ]
          ]
        ]
      }
    }
  ]
}

coords_area = area['features'][0]['geometry']['coordinates']
aoi_area = ee.Geometry.Polygon(coords_area)

station = [5.5730222, -0.24447074]

In [4]:
m = geemap.Map(zoom_start=6)
m.setCenter(23, 41)

#Load the SRTM image.
srtm = ee.Image('USGS/SRTMGL1_003');

#Apply slope algorithm to an image.
slope = ee.Terrain.slope(srtm);
sinImage = slope.divide(180).multiply(np.pi).sin();

visualization_params = {min: 0, max: 10000, 'palette': [
    '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
    'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
    '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
  ],};

m.addLayer(slope.select(0), visualization_params)

# marker = Marker(location=(40.9, 22.3), draggable=True)
# m.add_layer(marker);

m

Map(center=[41, 23], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [9]:
x = float(input('X: '))
y = float(input('Y: '))
xy = ee.Geometry.Point([x, y])
slope = slope.sample(xy, 30).first().get('slope').getInfo()

print('Slope (%):', slope)

X: 40
Y: 23
Slope (%): 1.0074855089187622


In [23]:
taskToexport = ee.batch.Export.image.toDrive(
        image = srtm,             
        region = aoi_area.bounds().getInfo()['coordinates'],
        description = 'Area',
        )

taskToexport.start()

In [27]:
taskToexport.status()

{'state': 'COMPLETED',
 'description': 'Area',
 'creation_timestamp_ms': 1616843384695,
 'update_timestamp_ms': 1616843470241,
 'start_timestamp_ms': 1616843460928,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'id': 'W3REHJYBION56WV34V4X4ZC2',
 'name': 'projects/earthengine-legacy/operations/W3REHJYBION56WV34V4X4ZC2'}

In [30]:
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
def lights(years):
    arr1 = ['F101992', 'F101993', 'F121994', 'F121995', 'F121996', 'F141997', 'F141998', 'F141999', 'F152000', 'F152001', 'F152002', 'F152003', 'F162004', 'F162005', 'F162006', 'F162007', 'F162008', 'F162009', 'F182010', 'F182011', 'F182012', 'F182013']
    arr2 = np.arange(1992, 2014)
    year_im1 = dict(zip(arr1, arr2))
    year_im12 = (list(year_im1.keys())[list(year_im1.values()).index(years)])
    # set our initial map parameters for Washington, DC
    center_lat = 39
    center_lon = 23.5
    zoomlevel = 6.3

    # initialize our map
    map1 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
    map1.add_basemap('Dark')
    naip_url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
    map1.add_wms_layer(url=naip_url, layers='0', name='NAIP Imagery', format='image/png', shown=True)
    url = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
    map1.add_tile_layer(url, name='Google Map', attribution='Google')
    visualization_params = {min: 0, max: 10000, 'palette': [
        'ffffcb', 'cc5500', 'cc5500'
      ],};
    # get 1996 composite, apply mask, and add as layer
    dmsp = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/{}".format(year_im12)).select('stable_lights')
    map1.addLayer(dmsp.mask(dmsp), visualization_params, "DMSP-OLS 1996")
    map1.addLayerControl()
    display(map1)

slider = widgets.Dropdown(
    options=np.arange(1992, 2014),
    value=1992,
    description='Year',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True)

interact(lights, years = slider);

interactive(children=(Dropdown(description='Year', options=(1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 20…